# Data Model

![mode.png](https://github.com/Cryogenic117/graph-summit-demo/blob/main/model.png?raw=true)

# Connect To Neo4j

In [114]:
from neo4j import GraphDatabase
import time
import ipywidgets as widgets
from IPython.display import display

URI = "bolt://localhost:7687"
AUTH = ("neo4j", "neo4j123")

# Create Constraints

In [115]:
airport_constraint = """
CREATE CONSTRAINT airport_unique IF NOT EXISTS
FOR (a:Airport)
REQUIRE a.iata IS UNIQUE;
"""

airline_constraint = """
CREATE CONSTRAINT airline_unique IF NOT EXISTS
FOR (a:Airline)
REQUIRE a.iata IS UNIQUE;
"""

route_constraint = """
CREATE CONSTRAINT route_node_key IF NOT EXISTS
FOR (r:Route)
REQUIRE (r.departure, r.arrival) IS NODE KEY;
"""

hotel_constraint = """
CREATE CONSTRAINT hotel_unique IF NOT EXISTS
FOR (h:Hotel)
REQUIRE h.code IS UNIQUE;
"""

# Create Indexes

In [116]:
hotel_index = """
CREATE INDEX IF NOT EXISTS FOR (h:Hotel) ON (h.country);
"""

airport_index = """
CREATE INDEX IF NOT EXISTS FOR (a:Airport) ON (a.country);
"""


# Load Airport Nodes

In [117]:

query_airports = """
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/Cryogenic117/graph-summit-demo/refs/heads/main/Data/df_airports.csv" AS row
WITH
  trim(row.Airport_IATA) AS iata,
  trim(row.City) AS city,
  trim(row.Country) AS country,
  point({latitude: toFloat(row.Latitude), longitude:  toFloat(row.Longitude)}) AS geolocation
MERGE (a:Airport {iata: iata})
ON CREATE SET
  a.city = city,
  a.country = country,
  a.geolocation = geolocation
RETURN DISTINCT 'Complete';
"""

# Load Route Nodes

In [118]:
query_routes = """
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/Cryogenic117/graph-summit-demo/refs/heads/main/Data/df_routes.csv" AS row
WITH
  trim(row.Departure) AS departure,
  trim(row.Arrival) AS arrival,
  toFloat(row.Distance_km) AS distance,
  toFloat(row.Duration_min) AS duration
MERGE (r:Route {departure: departure, arrival: arrival})
  ON CREATE SET r.distance = distance,
                r.duration = duration,
                r.id = randomUUID();
"""

# Load Airline Nodes

In [119]:
query_airlines = """
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/Cryogenic117/graph-summit-demo/refs/heads/main/Data/df_routes.csv" AS row
WITH trim(row.Airline_IATA) AS airline
MERGE (a:Airline {iata: airline});
"""

# Load Hotel Nodes

In [120]:
query_hotel = """
CALL() {
LOAD CSV WITH HEADERS FROM "https://drive.usercontent.google.com/u/0/uc?id=1EHRz56PWLdzJYQgTmBMVgZSjjFn_dmuA&export=download" AS row
  WITH
    toInteger(row.HotelCode) AS hotelCode,
    trim(row.countyName) AS county,
    trim(row.cityName) AS city,
    trim(row.HotelName) AS hotelName,
    toInteger(row.HotelRating) AS hotelRating,
    CASE
      WHEN row.Map IS NOT NULL AND row.Map <> ""
      THEN point({
            latitude: toFloat(split(row.Map, '|')[0]),
            longitude: toFloat(split(row.Map, '|')[1])
         })
      ELSE NULL
    END AS geoLocation,
    trim(row.HotelWebsiteUrl) AS website
  MERGE (h:Hotel {code: hotelCode})
  ON CREATE SET
    h.name = hotelName,
    h.rating = hotelRating,
    h.country = county,
    h.city = city,
    h.geolocation = geoLocation,
    h.website = CASE WHEN website IS NOT NULL AND website <> "" THEN website ELSE NULL END
  RETURN count(*) AS processed
} IN TRANSACTIONS
RETURN 'done' AS status;
"""

# Load Airline Properties

In [121]:
query_airline_props = """
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/Cryogenic117/graph-summit-demo/refs/heads/main/Data/airline_data.csv" AS row
WITH trim(row.IATA) AS iata,
     trim(row.Airline) AS airlineName,
     trim(row.`Country/Region`) AS region,
     trim(row.Comments) AS comments
MATCH (a:Airline {iata: iata})
SET a.name = CASE WHEN airlineName IS NOT NULL AND airlineName <> "" THEN airlineName ELSE a.name END,
    a.region = CASE WHEN region IS NOT NULL AND region <> "" THEN region ELSE a.region END,
    a.comments = CASE WHEN comments IS NOT NULL AND comments <> "" THEN comments ELSE a.comments END
"""

# Load Route Relationships

In [122]:
query_relationships = """
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/Cryogenic117/graph-summit-demo/refs/heads/main/Data/df_routes.csv" AS row
WITH
  trim(row.Departure) AS departure,
  trim(row.Arrival) AS arrival,
  trim(row.Airline_IATA) AS airline
MATCH (dep:Airport {iata: departure})
MATCH (arr:Airport {iata: arrival})
MATCH (r:Route {departure: departure, arrival: arrival})
MERGE (dep)-[:HAS_ROUTE]->(r)
MERGE (r)-[:DESTINATION]->(arr)
WITH departure, arrival, airline, r
MATCH (a:Airline {iata: airline})
MERGE (a)-[:OPERATES]->(r);
"""

# Load Hotel and Airport Relationship

Creates Near relationship for hotels and airports within 15 miles of each other

In [123]:
query_hotel_relationships = """
WITH 15 * 1609.34 / 111320.0 AS delta
MATCH (h:Hotel)
WHERE h.geolocation IS NOT NULL AND h.country IS NOT NULL
WITH h, h.geolocation AS loc, h.country as country, delta
MATCH (a:Airport)
WHERE a.geolocation IS NOT NULL
    AND a.country = country
  AND a.geolocation.latitude  >= loc.latitude  - delta
  AND a.geolocation.latitude  <= loc.latitude  + delta
  AND a.geolocation.longitude >= loc.longitude - delta
  AND a.geolocation.longitude <= loc.longitude + delta
MERGE (h)-[:NEAR]-(a)
"""

# Run Queries

In [124]:
queries = [
    ("Airport Constraint Creation", airport_constraint),
    ("Airline Constraint Creation", airline_constraint),
    ("Hotel Constraint Creation", hotel_constraint),
    ("Route Constraint Creation", route_constraint),
    ("Airport Geo Index Creation", airport_index),
    ("Hotel Geo Index Creation", hotel_index),
    ("Airport Nodes Creation", query_airports),
    ("Route Nodes Creation", query_routes),
    ("Airline Nodes Creation", query_airlines),
    ("Hotel Nodes Creation", query_hotel),
    ("Airline Properties Creation", query_airline_props),
    ("Route Relationships Creation", query_relationships),
    ("Hotel and Airport Relationship Creation", query_hotel_relationships),
]

progress_bar = widgets.IntProgress(value=0, min=0, max=len(queries))
status_label = widgets.Label(value="Starting...")
display(progress_bar, status_label)

driver = GraphDatabase.driver(URI, auth=AUTH)
with driver.session() as session:
    for i, (query_name, query) in enumerate(queries, start=1):
        status_label.value = f"Running {query_name}..."
        start_time = time.time()
        session.run(query)
        elapsed = time.time() - start_time
        status_label.value = f"Completed {query_name} in {elapsed:.2f} seconds."
        progress_bar.value = i
        time.sleep(1)
driver.close()

status_label.value = "All queries completed."

IntProgress(value=0, max=13)

Label(value='Starting...')